# Lezione 13

In [ ]:
import re

from liblet import Grammar, Derivation, Table, suffixes, closure, warn, HASH, ε

In [ ]:
def compute_εfirst(G):
  FIRST = Table(1, element = set)
  for t in G.T: FIRST[(t, )] = {t}
  FIRST[tuple()] = {ε}
  FIRST[(ε, )] = {ε}
  FIRST[(HASH, )] = {HASH}
  @closure
  def update_with_suffixes(FIRST):
    for N, α in G.P:
      FIRST[(N, )] |= FIRST[α]
      for γ in suffixes(α):
        A, *β = γ
        FIRST[γ] |= FIRST[(A, )] - {ε}
        if ε in FIRST[(A, )]: FIRST[γ] |= FIRST[β]
    return FIRST
  return update_with_suffixes(FIRST)

def compute_follow(G, FIRST):
  FOLLOW = Table(1, set)
  FOLLOW[G.S] |= {HASH}
  @closure
  def complete_follow(FOLLOW):
    for X, ω in G.P:
      for γ in suffixes(ω):
        A, *β = γ
        if A not in G.N: continue
        FOLLOW[A] |= FIRST[β] - {ε}
        if ε in FIRST[β]: FOLLOW[A] |= FOLLOW[X]
    return FOLLOW
  return complete_follow(FOLLOW)

def compute_table(G, FIRST, FOLLOW):
  TABLE = Table(2)
  for P in G.P:
    A, α = P
    for a in FIRST[α] - {'ε'}:
        TABLE[A, a] = P
    if 'ε' in FIRST[α]:
      for a in FOLLOW[A]:
          TABLE[A, a] = P
  return TABLE

# Simboli e input: stringhe e sequenze

Un "chiarimento" dovuto sull'uso dei termini *simboli* e *input* e sulle relative scelte implementative adottate per il software prodotto per questo corso.

In questi *handout*, i **simboli** (siano essi terminali che non) sono implementati come stringhe, che corrispondono al tipo [Text Sequence Type](https://docs.python.org/3.7/library/stdtypes.html#text-sequence-type-str) (brevemente `str`). Osservate che in Python *non esiste il tipo carattere*, pertanto anche quando i simboli sono composti da un'unica lettera, questi saranno di tipo `str`. 

Questo è evidente se, ad esempio, si ispeziona una `Grammar`. Prendiamo come esempio quella vista nella scorsa lezione:

In [ ]:
G = Grammar.from_string("""
Expr -> Term Expr′
Expr′ -> PLUS Term Expr′ | ε
Term -> Factor Factor′
Factor′ -> TIMES Factor Factor′ | ε
Factor -> LPAR Expr RPAR | NUMBER
""")
G.P

(Expr -> Term Expr′,
 Expr′ -> PLUS Term Expr′,
 Expr′ -> ε,
 Term -> Factor Factor′,
 Factor′ -> TIMES Factor Factor′,
 Factor′ -> ε,
 Factor -> LPAR Expr RPAR,
 Factor -> NUMBER)

In [ ]:
# Sono stringhe (elementi di tipo str) i non terminali

set(G.N)

{'Expr', 'Expr′', 'Factor', 'Factor′', 'Term'}

In [ ]:
# e anche i terminali!

set(G.T)

{'LPAR', 'NUMBER', 'PLUS', 'RPAR', 'TIMES'}

In base alla definizione di **linguaggio**, le **parole** e le **forme sentenziali** sono sequenze di simboli (sugli opportuni alfabeti). Ragion per cui l'implementazione naturale di tali entità è fatta usando [liste](https://docs.python.org/3.7/tutorial/datastructures.html#more-on-lists) o [tuple](https://docs.python.org/3.7/tutorial/datastructures.html#tuples-and-sequences).

Di nuovo, questo è evidente se, ad esempio, si ispeziona una forma sentenziale di una `Derivation`

In [ ]:
Derivation(G).leftmost((0, 3, 7, 5, 1)).sentential_form()

('NUMBER', 'PLUS', 'Term', 'Expr′')

In linea di principio, quindi, l'*input* (termine informale) che, dei nostri algoritmi, è costituito da una *parola* (termine formale) deve essere una sequenza di simboli, quindi una lista o tupla.

Non ha senso pensare all'*input* come a un elemento di tipo `str`, perché questo impedisce di riconoscere (in modo non "ambiguo") i simboli di cui è composto.

Prestate attenzione alla diversità tra seguenti asserzioni:
* `('NUMBER', 'NUMBER')` è una parola, ma non appartiene al linguaggio generato da `G`; 
* `'NUMBERNUMBER'` non solo non appartiene a tale linguaggio, ma non è nemmeno una parola sull'alfabeto `G.T` dal momento che, pur essendo un oggetto di tipo `str`, non è una sequenza di simboli terminali (contenuti in `G.T`).

## Se i simboli sono tutti lunghi uno?

Può accadere (ed è accaduto senza che lo rendessi esplicito, cosa che può avervi indotti in confusione), che se i simboli terminali sono **tutti** stringhe di lunghezza uno, l'*input* possa essere "impropriamente" implementato, invece che con una lista di stringhe di lunghezza uno, come una stringa. 

Questa "confusione" tra i tipi è resa possibile dal fatto che in Python l'espressione `INPUT[i]` è legittima sia nel caso in cui `INPUT` si riferisca a una stringa, che a una lista o a una tupla; inoltre nel caso in cui i simboli siano stringhe di lunghezza uno, le due espressiono hanno lo stesso valore.

In [ ]:
# implementazione propria

INPUT = ('p', 'i', 'p', 'p', 'o') # o analogamente list('pippo')

INPUT[1]

'i'

In [ ]:
# implementazione impropria

INPUT = 'pippo'

INPUT[1]

'i'

## Tokenizzazione e parsing

La confusione si può fare ancora più acuta quando parliamo di *tokenizzazione*, oltre che di parsing.

In tal caso ci sono due grammatiche (e quindi due linguaggi), in gioco.

La prima grammatica $G_t = (N_t, T_t, P_t, S_t)$ è quella del **tokenizzatore**, è in generale una grammatica regolare i cui terminali $T_t$ sono i caratteri dell'alfabeto di macchina (ad esempio i caratteri *Unicode*). 

Tale grammatica può essere pensata come l'unione di $k>0$ grammatiche regolari $G^k_t = (N^k_t, T_t, P^k_t, S^k_t)$ (in cui gli $N^k_t$ e $P^k_t$ sono disgiunti, gli $S^k_t$ sono distinti e 
i linguaggi $L(G^k_t)$ sono disgiunti al variare di $k$), ciascuna delle quali riconosce un certo "tipo" di *token*. $G_t$ è usualmente definita da $N_t = \{S_t\} \cup \bigcup N^k_t$ e $P_t =  \{S_t \to ( S^1_t | S^2_t |  \ldots | S^k_t )^* \} \cup \bigcup P^k_t$ (dove la produzione per il simbolo iniziale, qui descritta con la notazione impropria delle espressioni regolari, indica che le parole $G_t$ sono sequenze di parole delle varie $G^k_t$, ossia di token.

La seconda grammatica $G_p = (N_p, T_p, P_p, S_p)$ è quella del **parser**, è in generale una grammatica libera da contesto i cui terminali $T_p$ sono i simboli distinti delle grammatiche $G^k_t$, ossia $T_p = \{S^1_t, S^2_t, \ldots, S^k_t\}$. 

Rispetto alla situazione in cui si considera solo il parsing, in questo contesto parlare di *input* può condurre ad ancor maggior confusione: tale termine può infatti riferirsi sia alla parola (in $T_t^*$) data in input al tokenizzatore, che alla parola (in $T_p^*$) da esso restituita che sarà quindi data in input al parser.

### Un esempio concreto

Consideriamo come grammatica del parser quella vista nella precedente lezione, nella versione ripropsta all'inizio di questo handout; la grammatica del suo tokenizzatore è definita attraverso le espressioni regolari per `LPAR`, `PLUS`, `RPAR`, `TIMES` (che sono elementari, trattandosi di linguaggi finiti), e quella di `NUMBER` che è semplicemente `\d+`.

Per implementare il tokenizzatore seguiremo l'approccio basato sulle espressioni regolari visto in [L07](L07.ipynb); iniziamo con l'elenco di token e relative espressioni regolari

In [ ]:
EPXR_TOKEN2PATTERN = (
  ('LPAR',   r'\('),
  ('RPAR',   r'\)'),
  ('PLUS',   r'\+'),
  ('TIMES',  r'\*'),
  ('NUMBER', r'\d+')
)

Useremo due variabili globali `token` e `value` per immagazzinare il *look-ahead*; data la parola in input al tokenizzatore e l'elenco di espressioni regolari che definiscono la sua grammatica, la seguente funzione 

In [ ]:
def make_consume(word, token2pattern):
  tokens_re = re.compile('|'.join(f'(?P<{token}>{pattern})' for token, pattern in token2pattern))
  token_value_iterator = iter((token, value) for match in tokens_re.finditer(word) for token, value in match.groupdict().items() if value)
  def _advance():
    global token, value
    token, value = next(token_value_iterator, (HASH, None))
  _advance()
  def consume(expected):
    if token != expected:
      warn('Expected {}, found {}'.format(expected, token))
    else:
      _advance()
  return consume

consente di costruire la funzione `consume` che tenterà di consumare il token atteso e, qualora esso coincida con il look-ahead, avanzerà nella lettura aggiornando i valori di `token` e `value`.

Possiamo testarla ad esempio sull'input (`5`, `2`, `*`, `7`) sull'alfabeto Unicode.

In [ ]:
INPUT = '52 * 7'

consume = make_consume(INPUT, EPXR_TOKEN2PATTERN)

token, value

('NUMBER', '52')

In [ ]:
consume('NUMBER')

token, value

('TIMES', '*')

In [ ]:
consume('TIMES')

token, value

('NUMBER', '7')

Al parser non resta che determinare la derivazione che produce la sequenza di token visti:

In [ ]:
Derivation(G).leftmost((0, 3, 7, 4, 7, 5, 2))

Expr -> Term Expr′ -> Factor Factor′ Expr′ -> NUMBER Factor′ Expr′ -> NUMBER TIMES Factor Factor′ Expr′ -> NUMBER TIMES NUMBER Factor′ Expr′ -> NUMBER TIMES NUMBER Expr′ -> NUMBER TIMES NUMBER

# Parser ricorsivo discendente predittivo

Costruiamo ora un parser ricorsivo discendende (come quello sviluppato in [L09](L09.ipynb)) che usi le informazioni della tabella `TABLE` per decidere quali chiamate ricorsive fare (in funzione delle produzioni contenute nella tabella).

In questa prima implementazione non ci occupiano del valore dei token, l'unico obiettivo è determinare se l'espressione è valida.

Calcoliamo la `TABLE` che ci servirà per scrivere la procedure ricorsive.

In [ ]:
FIRST = compute_εfirst(G)
FOLLOW = compute_follow(G, FIRST)
compute_table(G, FIRST, FOLLOW)


Per ciascun non terminale, tramutiamo la riga relativa in una serie di chiamate ricorsive distinguendo tra le varie colonne grazie al *look-ahead*

In [ ]:
def parse_Expr():
  if token in {'LPAR', 'NUMBER'}:
    parse_Term()
    parse_Exprp()
  else:
    warn('Error parsing Expr')

In [ ]:
def parse_Exprp():
  if token in {'RPAR', HASH}:
    pass # questa è l'ε-produzione
  elif token == 'PLUS':
    consume('PLUS')
    parse_Term()
    parse_Exprp()
  else:
    warn('Error parsing Expr′')

In [ ]:
def parse_Term():
  if token in {'LPAR', 'NUMBER'}:
    parse_Factor()
    parse_Factorp()
  else:
    warn('Error parsing Term')

In [ ]:
def parse_Factorp():
  if token in {'PLUS', 'RPAR', HASH}:
    pass
  elif token == 'TIMES':
    consume('TIMES')
    parse_Factor()
    parse_Factorp()
  else:
    warn('Error parsing Factor′')

In [ ]:
def parse_Factor():
  if token == 'NUMBER':
    consume('NUMBER')
  elif token == 'LPAR':
    consume('LPAR')
    parse_Expr()
    consume('RPAR')
  else:
    warn('Error parsing Factor')

Possiamo collaudare il parser su una espressione valida e una contenente un errore

In [ ]:
INPUT = '52 * 7'

consume = make_consume(INPUT, EPXR_TOKEN2PATTERN)

parse_Expr() # non viene restituito nulla, ossia la parola è riconosciuta

In [ ]:
INPUT = '52 7'

consume = make_consume(INPUT, EPXR_TOKEN2PATTERN)

parse_Expr() # la parola non è una espressione valida, quindi viene restituito un errore

Error parsing Factor′
Error parsing Expr′


### <span style="color: red;">Esercizio</span>

Non è difficile scrivere un *generatore di parser* come abbiamo fatto nella lezione [L09](L09.ipynb); similmente si potrebbero modificare le procedure sopra riportate (o il generatore di parser) in modo che restituisca la derivazione ottenuta, sempre seguendo l'esmpio di quanto fatto nella lezione [L09](L13.ipynb).

## La valutazione di espressioni

Il discorso si complica un po' se volessimo ottenere il valore dell'espressione aritmetica, in quel caso l'atomo `NUMBER` andrebbe rimpiazzato da un numero e ciascuna procedura dovrebbe restituire il valore della sottoespressione di cui ha fatto il parsing.

In [ ]:
def eval_Expr():
  if token in {'LPAR', 'NUMBER'}:
    val = eval_Term()
    val += eval_Exprp()
    return val
  warn('Error parsing Expr')

def eval_Exprp():
  if token in {'RPAR', HASH}:
    return 0 # questa è l'unità addittiva
  elif token == 'PLUS':
    consume('PLUS')
    val = eval_Term()
    val += eval_Exprp()
    return val
  warn('Error parsing Expr′')

def eval_Term():
  if token in {'LPAR', 'NUMBER'}:
    val = eval_Factor()
    val *= eval_Factorp()
    return val
  warn('Error parsing Term')

def eval_Factorp():
  if token in {'PLUS', 'RPAR', HASH}:
    return 1 # questa è l'unità moltiplicativa
  elif token == 'TIMES':
    consume('TIMES')
    val = eval_Factor()
    val *= eval_Factorp()
    return val
  warn('Error parsing Factor′')

def eval_Factor():
  if token == 'NUMBER':
    val = int(value) # la variabile globale modificata da consume
    consume('NUMBER')
    return val
  elif token == 'LPAR':
    consume('LPAR')
    val = eval_Expr()
    consume('RPAR')
    return val
  warn('Error parsing Factor')

Siamo pronti ad usare queste funzioni per valutare le espressioni

In [ ]:
INPUT = '2 * 3 + 4'

consume = make_consume(INPUT, EPXR_TOKEN2PATTERN)

eval_Expr()

10

## Extended CFG

Se con la notazione $\left\{\alpha\right\}^*$ indichiamo 0 o più ripetizioni di $\alpha$, la precedene grammatica si può intendere come

$E \to T \left\{+ T\right\}^* \\
T \to F \left\{* F\right\}^* \\
F \to ( E ) | i$

il che suggerisce l'uso di un cicli `while` (invece della ricorsione di coda) per gestire la ripetizione.

In [ ]:
def eval_Expr():
  if token in {'LPAR', 'NUMBER'}:
    val = eval_Term()
    while token == 'PLUS':
      consume('PLUS')
      val += eval_Term()
    if token in {'RPAR', HASH}: return val
  warn('Error parsing Expr')

def eval_Term():
  if token in {'LPAR', 'NUMBER'}:
    val = eval_Factor()
    while token == 'TIMES':
      consume('TIMES')
      val *= eval_Factor()
    if token in {'PLUS', 'RPAR', HASH}: return val  
  warn('Error parsing Term')

Un ultimo test suggerisce che l'approccio funziona!

In [ ]:
INPUT = '2 * 3 * 4 + 5 * ( 6 + 7 )'

consume = make_consume(INPUT, EPXR_TOKEN2PATTERN)

eval_Expr()

89

Ma restsa un problemino con `)`, perché? come potrebbe essere risolto?

In [ ]:
INPUT = '2 )'

consume = make_consume(INPUT, EPXR_TOKEN2PATTERN)

eval_Expr()

2